<a href="https://colab.research.google.com/github/VolkhinD/Steel/blob/main/Ultralytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install ultralytics
!pip install PyYAML

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow # since cv2.imshow collaps Colab session
from sklearn.model_selection import GroupShuffleSplit
import torch
import seaborn as sns

import os
import shutil
import yaml

sns.set_palette('BrBG_r')

In [ ]:
IMG_SIZE = (256, 1600)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Play with YOLO

In [ ]:
img1 = '/content/drive/MyDrive/Data/IMG_1541.jpg'
img2 = '/content/drive/MyDrive/Data/IMG_1476.jpg'
img3 = '/content/pr_cat.jpg'
im1 = cv2.imread(img1)

In [ ]:
seg_model = YOLO('yolov8m-seg.pt')

In [ ]:
results = seg_model.predict(im1)
result = results[0]
masks = result.masks
len(masks)

In [ ]:
mask1 = masks[0].data.cpu()
mask2 = masks[1].data.cpu()

In [ ]:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(im1)
ax[1].imshow(mask1.permute(1, 2, 0))
ax[2].imshow(mask2.permute(1, 2, 0))

# Create masks and polygons

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data/Steel/train.csv')


In [ ]:
data.ClassId = data.ClassId.map({1: 0, 2: 1, 3: 2, 4: 3})
data.head()

In [ ]:
def rle_2D_mask(idx, imgshape=IMG_SIZE):

  """Creates mask for all types of defects"""

  width = imgshape[0]
  height= imgshape[1]
  img_name = data.iloc[idx].ImageId
  all_img = data.groupby('ImageId').get_group(img_name)

  mask = np.zeros(width*height).astype(np.uint8)

  if all_img.EncodedPixels.isnull().any(): # any because I plan to add img with no defects so they have only 'ImageId'
    return mask.reshape(width, height)

  for class_id, rle in zip(all_img.ClassId.to_numpy(), all_img.EncodedPixels.to_numpy()):
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = class_id


  return np.flipud(np.rot90(mask.reshape(height, width), k=1))

In [ ]:
""" Copied from Kaggle"""
def rle2mask(rle, imgshape=IMG_SIZE):

  """ Creates mask for only one type of deffect """

  width = imgshape[0]
  height= imgshape[1]

  mask = np.zeros( width*height ).astype(np.uint8)

  array = np.asarray([int(x) for x in rle.split()])
  starts = array[0::2]
  lengths = array[1::2]

  for index, start in enumerate(starts):
      mask[int(start):int(start+lengths[index])] = 1


  return np.flipud(np.rot90( mask.reshape(height, width), k=1))

In [ ]:
path = '/content/drive/MyDrive/Data/Steel/data/train/images/'
fn = data['ImageId'].iloc[2]
img = cv2.imread(path + fn)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
mask1 = rle_2D_mask(2)
plt.imshow(mask1)

## RLE to Polygon

In [ ]:
fn = data['ImageId'].iloc[2]
img = cv2.imread(path + fn)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
contours, heirarchy = cv2.findContours(mask1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#draw the obtained contour lines(or the set of coordinates forming a line) on the original image
cv2.drawContours(img, contours, -1, (0,255,0), 3)
#show the image
cv2_imshow(img)


In [ ]:
print(f"There are {len(contours)} objects on this example mask")
print(f"Image shape {img.shape}")

In [ ]:
# f = open("/content/drive/MyDrive/Data/Steel/data/train/labels/test.txt", "a")
# for i in range(10):
#   f.write(str(i) + '\t')
# f.write('\n')
# f.close()



# Getting Ready Images with [this Artical](https://dev.to/andreygermanov/how-to-implement-instance-segmentation-using-yolov8-neural-network-3if9#get_started)

1. Created a folder for dataset and two subfolders in it: "images" and "labels".

2. Put the images to the "images" subfolder.

3. For each image, create an annotation text file in the "labels" subfolder. Annotation text files should have the same names as image files and the ".txt" extensions. In annotation file you should add records about each object, that exist on the appropriate image in the following format:

    {object_class_id} {polygon}

    **object_class_id** is a label of object class, there are 4 classes in dataset

    **polygon** is a coordinates of bounding polygon for this object in the following format: x1 y1 x2 y2 ...

    Coordinates should be normalized to fit in a range from 0 to 1. To calculate them, I need to use the following formulas:

    x = x/image_width
    y = y/image_height

4. Finally, I need to create a dataset descriptor YAML-file, that points to created datasets and describes the object classes in them.



## Split to Train and Validation

In [ ]:
"""Done this ones """

splitter = GroupShuffleSplit(test_size=.15, n_splits=2, random_state = 7)
split = splitter.split(data, groups=data.ImageId)
train_inds, test_inds = next(split)

# train = data.iloc[train_inds]
# test = data.iloc[test_inds]

print(f"Length of train set is {len(train_inds)}")
print(f"Length of test set is {len(test_inds)}")


In [ ]:
def transfer_to_fol(source, destination, indexes):
  absent_files = []
  for idx in indexes:
    img_name = data.ImageId.iloc[idx]
    src_path = os.path.join(source, img_name)
    dst_path = os.path.join(destination, img_name)
    try:
      shutil.move(src_path, dst_path)
    except:
      absent_files.append(img_name)

  return absent_files

In [ ]:
# source = '/content/drive/MyDrive/Data/Steel/all/images'
# destination_train = '/content/drive/MyDrive/Data/Steel/data/train/images'
# destination_val = '/content/drive/MyDrive/Data/Steel/data/val/images'
# train_absent = transfer_to_fol(source, destination_train, train_inds)
# test_absent = transfer_to_fol(source, destination_val, test_inds)

In [ ]:
print('Number of absent files in train 371')
print('Number of absent files in test 58')
print(f"Number of Images absent in dataset {len(os.listdir('/content/drive/MyDrive/Data/Steel/all/images'))}")

## Create Text Files

from documentation:

**Each segmentation label must have a minimum of 3 xy points**

**Labels should start with 0**


In [ ]:
def create_txt_file(idx, path):
  img_name = data['ImageId'].iloc[idx]
  img_class = data['ClassId'].iloc[idx]
  rle = data['EncodedPixels'].iloc[idx]
  mask = rle2mask(rle)
  contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  f = open(path + img_name[:-3] + 'txt', "a")
  for pair in contours:
    pair = pair.reshape(-1, 2)
    if len(pair) > 2:
      f.write(str(img_class) + ' ')
      for x, y in pair:
        x, y = x / 1600, y / 256
        f.write(str(x) + ' ' + str(y) + ' ')
      f.write('\n')
  f.write('\n')
  f.close()

In [ ]:
train_path = '/content/drive/MyDrive/Data/Steel/data/train/labels/'
val_path = '/content/drive/MyDrive/Data/Steel/data/val/labels/'

# existed_train = set()
# existed_val = set()

# train_all_img = os.listdir('/content/drive/MyDrive/Data/Steel/data/train/images/')
# val_all_img = os.listdir('/content/drive/MyDrive/Data/Steel/data/val/images/')

# for img_name in train_all_img:
#     if img_name not in existed_train:
#         existed_train.add(img_name)
#         for idx in data.groupby(data.ImageId).get_group(img_name).index:
#             create_txt_file(idx, train_path)
# for img_name in val_all_img:
#     if img_name not in existed_val:
#         existed_val.add(img_name)
#         for idx in data.groupby(data.ImageId).get_group(img_name).index:
#             create_txt_file(idx, val_path)

In [ ]:
assert len(os.listdir(val_path)) == len(os.listdir('/content/drive/MyDrive/Data/Steel/data/val/images'))
assert len(os.listdir(train_path)) == len(os.listdir('/content/drive/MyDrive/Data/Steel/data/train/images'))

## Create YAML-file

In [ ]:
data =  dict(
    train = '/content/drive/MyDrive/Data/Steel/data/train/images',
    val =  '/content/drive/MyDrive/Data/Steel/data/val/images',
    nc = 4,
    names = ['0', '1', '2', '3']
            )
with open('parameters.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

# Model Training

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir path/to/runs

In [ ]:
model = YOLO('yolov8m-seg.pt')
model = model.to(DEVICE)

In [ ]:
model.train(data='parameters.yaml', epochs=30)

**box_loss** shows the amount of error in detected bounding boxes.

**cls_loss** shows the amount of error in detected object classes.

**seg_loss** shows the amount of error in detected segmentation masks

***DFL*** contribution to Bbox loss: Distance-IoU (DIoU) and Complete IoU (CIOU) are relatively recent adaptions to traditional Intersection over Union (IoU) loss used in the YOLOv3, and Directional Feature Learning (DFL) is the method used to train them. When DFL is applied to CIOU, we call it DFL-CIOU. DFL provides gradients that can guide the learning of boundary predicted features, thus helping to reduce the bbox loss.
Decrease in DFL: If DFL decreases during training, it means the model is getting better at predicting the boundary box for your target detection task. The underlying reason for this decrease is indeed that the difference (or "loss") between the ground truth bounding boxes and the model's predicted bounding boxes is reducing over time.


Why the loss split to several metrics? Because the model could correctly detect the bounding box around the object, but incorrectly detect the object class in this box. For example, in my practice, it detected the dog as a horse, but the dimensions of the object were detected correctly.

## What is Mean Average Precision (mAP)?

mAP formula is based on the following sub metrics:

- Confusion Matrix,
- Intersection over Union(IoU),
- Recall,
-

Here is a summary of the steps to calculate the AP:

1. Generate the prediction scores using the model.
2. Convert the prediction scores to class labels.
3. Calculate the confusion matrix—TP, FP, TN, FN.
4. Calculate the precision and recall metrics.
5. Calculate the area under the precision-recall curve.
6. Measure the average precision.



# Save Weights

In [ ]:
sourse_dir = '/content/runs/segment/train3/weights/best.pt'
dest_dir = '/content/drive/MyDrive/Models/yolo_steel.pt'
shutil.copy(sourse_dir, dest_dir)

# Evaluation

In [ ]:
m = '/content/runs/segment/train3/confusion_matrix.png'
img = cv2.imread(m)
plt.figure(figsize=(12, 12))
plt.imshow(img)
plt.show()

In [ ]:
r = pd.read_csv('/content/runs/segment/train3/results.csv')
r.head()

In [ ]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(1, 1), edgecolor = (0.0, 0.0, 0.0, 0.0))
fig.suptitle("Precision and Recal Rise though Traing")
sns.relplot(data=r, x=r.iloc[:, 0], y=r.iloc[:, 5])
sns.relplot(data=r, x=r.iloc[:, 0], y=r.iloc[:, 6])